# Ray Tune - Understanding Hyperparameter Tuning

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)

Many of the [Ray RLlib](../ray-rllib/00-Ray-RLlib-Overview.ipynb) lessons used [Ray Tune](http://tune.io) to train policies. This meant we trained _parameters_ that defined the policies. Now we'll learn that Ray Tune was actually designed to determine the best _hyperparameters_ for the problem, before training to determine the _parameters_.

This lesson introduces the concepts of _Hyperparameter Tuning or Optimization_ (HPO) and works through a nontrivial example using Tune. 

See also the [Hyperparameter Tuning References](References-Hyperparameter-Tuning.ipynb) notebook and the [Tune documentation](http://tune.io), in particular, the [API reference](https://docs.ray.io/en/latest/tune/api_docs/overview.html).

## What Are Hyperparameters?

In _supervised learning_, we train a model with labeled data so the model can properly label new data values. Everything about the model is defined by a set of _parameters_, such as the weights in a linear regression. 

In contrast, the _hyperparameters_<sup>1</sup> define structural details about the kind of model itself, like whether or not we are using a linear regression or what architecture is best for a neural network, etc. Other quantities considered hyperparameters include learning rates, discount rates, etc. If we want our training process and resulting model to work well, we first need to determine the optimal or near-optimal set of hyperparameters.

How do we determine the optimal hyperparameters? The most straightfoward approach is to perform a loop where we pick a candidate set of values from some reasonably inclusive list of possible values, train a model, compare the results achieved with previous loop iterations, and pick the set that performed best. This process is called _Hyperparameter Tuning_ or _Optimization_ (HPO).

This simple algorithm can quickly become very expensive, however. Training a single neural networks can be compute intensive and the space of all possible architectures is huge. Hence, much of the research in hyperparameter tuning, especially for neural networks, focuses on ways to optimize HPO, such as early stopping and pruning the search space when some combinations appear to perform poorly.

1. _Hyperparameter_ is often spelled _hyper parameter_, but we'll use the spelling with no space.

## A Simple Example: $k$-Means 

Let's start with a very simple example of HPO, finding $k$ in $k$-means. 

The $k$-means algorithm finds clusters in a data set. It's a canonical example of _unsupervised learning_, where information is extracted from a data set, rather than using labeled data to train a model for labelling new data, as in _supervised learning_. We won't discuss the algorithm details, but the essense of it involves a "guess" for the expected number of clusters, the $k$ value, then calculating $k$ centroids (the coordinates at the center), one per cluster, along with determining to which cluster each data point belongs. The details are in [$k$-means Wikipedia article](https://en.wikipedia.org/wiki/K-means_clustering). The following animation shows the algorithm in action for a two-dimensional data set where three clusters are evident.

![K-Means Convergence](../images/tune/K-means_convergence.gif)

(source: [Wikipedia](https://en.wikipedia.org/wiki/K-means_clustering). [Larger Image](https://en.wikipedia.org/wiki/K-means_clustering#/media/File:K-means_convergence.gif))

While it is easy to see the clusters in this two-dimensional data set, that won't be for arbitrary datasets, especially those with more than three dimensions. Hence, we should determine the best $k$ value by trying many values and picking the value that appears to be best. In this case, "best" would mean that we minimize the distances between the datapoints and centroids. 

With just one hyperparameter, this problem is comparatively simple and brute force calculations to find the optimal $k$ is usually good enough.

## HPO for Neural Networks

Where HPO really becomes a challenge is finding the right neural network architecture for your problem. Why are neural networks a challenge. Consider this image of a typical architecture:

![Typical Neural Network](../images/tune/hpo-neural-network-example.png)

Every number you see is a hyperparameter! So are the decisions about how many layers to have, what kind of layer to use for each layer, etc. The space of possible hyperparameters is enormous, too big to explore naively.

So called _neural architecture search_ (NAS) has become a research field in its own right, along with general research in optimizing HPO. 

## Using Ray Tune

We used Ray Tune in several of the reinforcement learning lessons, where we actually didn't optimize any hyperparameters, we just used it to drive the RLlib training process. Now we'll see an example of HPO with Tune.

First, we want to make sure that Ray is running and we want to initialize Ray in this "driver" program explicitly. 

In [ ]:
import ray
from ray import tune

The following shell command will probably print _INFO: Ray is already running._ If not, follow the instructions it prints to start Ray.

In [ ]:
!../tools/start-ray.sh --check --verbose

Now we start Ray in this "driver" program (notebook).

> **Note:** The call to `tune.run` below would handle Ray initialization for us, if we omitted the `ray_auto_init=False` option, but that would lead to multiple Ray clusters running. So, we prevent Tune from doing this for us and we initialize Ray explicitly ourselves.

In [ ]:
ray.init(address='auto', ignore_reinit_error=True)

In our [RLlib Simple Multi-Armed Bandit](../ray-rllib/multi-armed-bandits/03-Simple-Multi-Armed-Bandit.ipynb) lesson, we used Tune to train RLlib, but not train hyperparameters. 

Now we'll use Tune to train hyperparameters. We'll use the same experimental environment we used in our [RLlib _CartPole_ lesson](../ray-rllib/explore-rllib/01-Application-Cart-Pole.ipynb). 

> **Aside:** In case you haven't gone through the [Ray RLlib tutorial](../ray-rllib/00-Ray-RLlib-Overview.ipynb), [CartPole](https://gym.openai.com/envs/CartPole-v1/) is an [OpenAI Gym](https://gym.openai.com/envs/) environment that simulates a cart that moves left or right while attempting to balance a vertical pole. A _policy_ is trained to optimize changing the velocity up and down to control right and left movement with the goal of keeping the pole balanced for as long as possible.

The most important hyperparameters for _CartPole_ are for the neural network used to learn how to balance the pole. A simple network suffices. We'll use a fully-connected network with two hidden layers, but use Tune to find an optimal choice for the sizes of the layers. To keep the computation tractable for our purposes, we'll just pick sizes from the list of 20 and 40. 

It's probably that some specific number between these 10 and 100 is optimal for one layer and a different number is optimal for the other layer. However, the computation required for trying all possible combinations is $O(n^2)$. You should consider trying more numbers if you don't mind waiting!

However, the two we've chosen are good enough as we'll see. We'll consider what "good enough" means when we look at the results.

The next cell runs Tune for this purpose. The comments explain what each argument does. We'll do four tries, one for each combination of the two possible values for the two hidden layers.

In [ ]:
analysis = tune.run(
    "PPO",                                    # Use proximal policy optimization to train 
    stop={"episode_reward_mean": 400},        # Stopping criteria, when average reward over the episodes
                                              # of training equals 400 out of a maximum possible 500 score.
    config={
        "env": "CartPole-v1",                 # Tune can associate this string with the environment.
        "num_gpus": 0,                        # If you have GPUs, go for it!
        "num_workers": 6,                     # Number of Ray workers to use (arbitrary choice).
        "model": {                            # The NN model we'll optimize.
            'fcnet_hiddens': [                # "Fully-connected network with N hidden layers".
                tune.grid_search([20, 40]),   # Try these four values for layer one.
                tune.grid_search([20, 40])    # Try these four values for layer one.
            ]
        },
        "eager": False,                       # Flag for TensorFlow; don't use eager evaluation.
    },
    verbose=2,                                # Change to 0 or 1 to reduce the output.
    ray_auto_init=False                       # Don't allow Tune to initialize Ray.
)

## Understanding the Results

Which one performed best based on our stopping criteria?

In [ ]:
analysis.get_best_config(metric="episode_reward_mean")

So, the smallest combination is good enough, even the best according to this metric! 

Let's look at the data for all of them:

In [ ]:
df = analysis.dataframe()
df

As stated above, all four combinations actually work about equally well, as far as the `episode_reward_mean` mean is concerned. So what's actually best? What's "good enough" in this case?

It's useful to consider the `training_iteration` (roughly proportional to `episodes_total`) and `timesteps_total`. We see from this table that the `[20,20]` hyperparameter set took the *most* training iterations, which is understandable as it is the least powerful network configuration. The corresponding number of timesteps was the longest. In contrast, `[40,40]` was the fastest to train with almost the same `episode_reward_mean` value.

Since all four combinations perform equally well, perhaps it's best to choose the largest network as it trains the fastest. If we need to train the neural network frequently, then fast training times might be most important. This also suggests that we should be sure the trial sizes we used are really best. In a real-world application, you would want to spend more time on HPO, trying a larger set of possible values.

## Analyzing the Results with TensorBoard 

Here is the directory where the training data was written for the "best" hyperparameter set:

In [ ]:
analysis.get_best_logdir(metric="episode_reward_mean")

There is a separate directory for each trial run.

The easiest way to inspect all the training data, including graphs, is to use [TensorBoard](https://www.tensorflow.org/tensorboard).

1. If you are runnng on the Anyscale Platform, click the _TensorBoard_ link. 
2. If you running this notebook on a laptop, open a terminal window using the `+` under the _Edit_ menu, run the following command, then open the URL shown in the output.

```
tensorboard --logdir ~/ray_results 
```

Here is a screen shot of TensorBoard after the previous Tune run, showing how _scalars_ like the `episdoe_reward_mean` evolved. Some data shown is for runs with trial values of 60 and 80:

![TensorBoard](../images/tune/TensorBoard-CartPole-HPO-scalars.png)

Here is a table of the hyperparameter data, similar to what we saw above by looking at the `analysis.dataframe()`:

![TensorBoard](../images/tune/TensorBoard-CartPole-HPO-hyperparameters.png)

## Other Useful Information 

The `ray.tune.trial.Trial` object ([documentation](https://docs.ray.io/en/latest/tune/api_docs/internals.html?highlight=Trial#trial-docstring)) records lots of information about particular trials:

In [ ]:
trial = analysis.get_best_trial(metric="episode_reward_mean")
t, type(t)

In [ ]:
t.stopping_criterion

In [ ]:
t.metric_analysis

How long did it take??

In [ ]:
stats = analysis.stats()
stats

In [ ]:
print(f'{stats["timestamp"] - stats["start_time"]} seconds')

In [ ]:
analysis.runner_data()

(pid=95600) Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz to ../data/mnist/MNIST/raw/train-images-idx3-ubyte.gz
0it [00:00, ?it/s]0) 
(pid=95599) 2020-07-18 12:07:12,544	ERROR function_runner.py:115 -- Runner Thread raised error.
(pid=95599) Traceback (most recent call last):
(pid=95599)   File "/Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/ray/tune/function_runner.py", line 109, in run
(pid=95599)     self._entrypoint()
(pid=95599)   File "/Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/ray/tune/function_runner.py", line 163, in entrypoint
(pid=95599)     return self._trainable_func(config, self._status_reporter)
(pid=95599)   File "/Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/ray/tune/function_runner.py", line 273, in _trainable_func
(pid=95599)     output = train_func(config)
(pid=95599)   File "<ipython-input-11-17c3bbc13f17>", line 3, in trainer
(pid=95599)  

2020-07-18 12:34:49,127	INFO (unknown file):0 -- gc.collect() freed 60 refs in 1.0079413270013902 seconds


(pid=2344) 2020-07-18 12:34:49,710	ERROR function_runner.py:115 -- Runner Thread raised error.
(pid=2344) Traceback (most recent call last):
(pid=2344)   File "/Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/ray/tune/function_runner.py", line 109, in run
(pid=2344)     self._entrypoint()
(pid=2344)   File "/Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/ray/tune/function_runner.py", line 163, in entrypoint
(pid=2344)     return self._trainable_func(config, self._status_reporter)
(pid=2344)   File "/Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/ray/tune/function_runner.py", line 273, in _trainable_func
(pid=2344)     output = train_func(config)
(pid=2344)   File "<ipython-input-11-17c3bbc13f17>", line 3, in trainer
(pid=2344) TypeError: get_data_loaders() got an unexpected keyword argument 'data_root'
(pid=2344) Exception in thread Thread-1:
(pid=2344) Traceback (most recent call last):
(pid=23

We used grid search here, which is a naïve approach. In subsequent lessons, we'll explore how to optimize the search process using some of Tune's built-in algorithms for this purpose.